<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Notion - Get page
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Notion/Notion_Get_page.ipynb" target="_parent"><img src="https://img.shields.io/badge/-Open%20in%20Naas-success?labelColor=000000&logo="/></a>

## Input

### Import library

In [64]:
import requests
import pandas as pd
import json
from pprint import pprint
from datetime import datetime

### Variables

In [65]:
# Enter Notion token API
TOKEN_API = 'secret_gF6bJPSyOgt5oZgb2sgT1yiMxfS4LqNmWmd2M8S5vzl'

# Enter page url
PAGE_URL = 'https://www.notion.so/naas-official/OKR-Q3-2021-d34d4a58b4d144e59b5218866958cdea'

## Model

### Get properties

In [17]:
# make a request to Notion API and receive a Python dictionary
def fetch_raw_properties(token_api, page_url):
    page_id = get_id_from_url(page_url)
    url = f'https://api.notion.com/v1/pages/{page_id}'
    headers = create_headers(token_api, _VERSION)
    res = requests.get(url, headers=headers)
    try:
        res.raise_for_status()
    except requests.HTTPError as e:
        return e
    return res.json()

page = fetch_raw_properties(TOKEN_API, PAGE_URL)
pprint(page)

{'archived': False,
 'created_time': '2021-07-08T12:55:00.000Z',
 'id': 'd34d4a58-b4d1-44e5-9b52-18866958cdea',
 'last_edited_time': '2021-08-23T11:36:00.000Z',
 'object': 'page',
 'parent': {'page_id': '89cc293f-4fee-4554-85d2-61bc7b845d6b',
            'type': 'page_id'},
 'properties': {'title': {'id': 'title',
                          'title': [{'annotations': {'bold': False,
                                                     'code': False,
                                                     'color': 'default',
                                                     'italic': False,
                                                     'strikethrough': False,
                                                     'underline': False},
                                     'href': None,
                                     'plain_text': 'OKR Q3 2021',
                                     'text': {'content': 'OKR Q3 2021',
                                              'link': None},
    

### Get content
👉  The content of a page is return as a array of blocks by the Notion API 
```json
{
  "object": "block",
  "id": "9bc30ad4-9373-46a5-84ab-0a7845ee52e6",
  "created_time": "2021-03-16T16:31:00.000Z",
  "last_edited_time": "2021-03-16T16:32:00.000Z",
  "has_children": false,
  "type": "to_do",
  "to_do": {
    "text": [
      {
        "type": "text",
        "text": {
          "content": "Lacinato kale",
          "link": null
        },
        "annotations": {
          "bold": false,
          "italic": false,
          "strikethrough": false,
          "underline": false,
          "code": false,
          "color": "default"
        },
        "plain_text": "Lacinato kale",
        "href": null
      }
    ],
    "checked": false
  }
}
```

Each block is a dictionary with different keys:
- id *(str)*
- has_children *(bool)*
- created_time *(str)*
- last_edited_time *(str)*
- type *(str)*
- {type} *(dict)*



{type} is an object with type-specific block information<br>
List of block type:
- paragraph
- heading(1,2,3)
- bullet list item
- numbered list item
- to_do_blocks
- toggle block
- child page block

More info here: https://developers.notion.com/reference/block 

🚨 BEAWARE OF:
- I can't retreive the children element of a block: it's not the same behaviour than the one in Block Object look like it's a bug from the API 
- Some data information are lost. Exemple: the color of the text and the link 
- blank blocks are count as a paragraph we maybe need to create a new category for them or delete them from the result

In [62]:
def fetch_raw_blocks(token_api, page_url):
    page_id = get_id_from_url(page_url)
    url = f'https://api.notion.com/v1/blocks/{page_id}/children'
    headers = create_headers(token_api, _VERSION)
    response = requests.get(url, headers=headers)
    res = requests.get(url, headers=headers)
    try:
        res.raise_for_status()
    except requests.HTTPError as e:
        return e
    return res.json()['results']

blocks = fetch_raw_blocks(TOKEN_API, PAGE_URL)
pprint(blocks[0])

{'created_time': '2021-07-08T12:55:00.000Z',
 'has_children': False,
 'id': 'edd11e77-95dd-4fa7-a0fc-a421f8ad3710',
 'last_edited_time': '2021-07-09T13:37:00.000Z',
 'object': 'block',
 'paragraph': {'text': [{'annotations': {'bold': True,
                                         'code': False,
                                         'color': 'default',
                                         'italic': False,
                                         'strikethrough': False,
                                         'underline': True},
                         'href': None,
                         'plain_text': 'Acteur 1: les entreprises',
                         'text': {'content': 'Acteur 1: les entreprises',
                                  'link': None},
                         'type': 'text'}]},
 'type': 'paragraph'}


### Functions

In [124]:
_VERSION = '2021-08-16'

headers = {'Authorization': f'Bearer {TOKEN_API}',
           'Notion-Version': f'{_VERSION}'}

def get_id_from_url(url):
    return url.split('-')[-1]

def get(url):
    res = requests.get(url, headers=headers)
    try:
        res.raise_for_status()
    except requests.HTTPError as e:
        return e
    return res.json()

def get_content(page_url):
    # Get id from page
    page_id = get_id_from_url(page_url)
    url = f'https://api.notion.com/v1/blocks/{page_id}/children'
    
    # Get json
    res_json = get(url)
    blocks = res_json.get("results")

    # Parse json
    contents = []
    for block in blocks:
        content = {
            "object": block.get("object"),
            "id": block.get("id"),
            "type": block.get("type"),
            "has_children": block.get("has_children"),
            "created_time": datetime.strptime(block.get("created_time"), '%Y-%m-%dT%H:%M:%S.000Z'),
            "last_edited_time": datetime.strptime(block.get("last_edited_time"), '%Y-%m-%dT%H:%M:%S.000Z'),
        }
        type_contents = block.get(block.get("type")).get("text")
        if type_contents is not None:
            for t in type_contents:
                annotations = t.get("annotations")
                content.update(annotations)
                text = t.get("text")
                content.update(text)
        contents.append(content)
    
    df = pd.DataFrame(contents)
    return df

## Output

### Get properties : Table format

In [113]:
df_properties = get_properties(PAGE_URL)
df_properties

{'object': 'page',
 'id': 'f59c898f-5ad3-4587-a544-f9e06b45be5f',
 'created_time': '2021-05-18T17:17:00.000Z',
 'last_edited_time': '2021-08-23T13:11:00.000Z',
 'parent': {'type': 'database_id',
  'database_id': '8910d64d-e001-479c-8494-fbecbf52b525'},
 'archived': False,
 'properties': {'Last Edited Time': {'id': '0AiB',
   'type': 'last_edited_time',
   'last_edited_time': '2021-08-23T13:11:00.000Z'},
  'Created By': {'id': 'F%5D)%3F',
   'type': 'created_by',
   'created_by': {'object': 'user',
    'id': '6e3bab71-beeb-484b-af99-ea30fdef4773',
    'name': 'Jeremy Ravenel',
    'avatar_url': 'https://lh3.googleusercontent.com/a-/AOh14Ghzj2FjcyVMNQm8vOmap4GNAmairAxAMz0QJXww=s100',
    'type': 'person',
    'person': {'email': 'jeremy.ravenel@cashstory.com'}}},
  'Created': {'id': 'Ird4',
   'type': 'created_time',
   'created_time': '2021-05-18T17:17:00.000Z'},
  'Meeting Date': {'id': 'Q%5B%3Dk',
   'type': 'date',
   'date': {'start': '2021-05-23', 'end': None}},
  'Type': {'id': '_

In [123]:
PAGE_URL = "https://www.notion.so/naas-official/Weekly-Sync-20-05-2021-f59c898f5ad34587a544f9e06b45be5f"

# Get id from page
page_id = get_id_from_url(PAGE_URL)
url = f'https://api.notion.com/v1/pages/{page_id}'

# Get json
res_json = get(url)

# Parse json
content = {
    "object": res_json.get("object"),
    "id": res_json.get("id"),
    "url": res_json.get("url"),
    "archived": res_json.get("archived"),
    "created_time": datetime.strptime(res_json.get("created_time"), '%Y-%m-%dT%H:%M:%S.000Z'),
    "last_edited_time": datetime.strptime(res_json.get("last_edited_time"), '%Y-%m-%dT%H:%M:%S.000Z'),
}
properties = res_json.get("properties")
pprint(properties)
for p in properties:
    print(p)

{'Created': {'created_time': '2021-05-18T17:17:00.000Z',
             'id': 'Ird4',
             'type': 'created_time'},
 'Created By': {'created_by': {'avatar_url': 'https://lh3.googleusercontent.com/a-/AOh14Ghzj2FjcyVMNQm8vOmap4GNAmairAxAMz0QJXww=s100',
                               'id': '6e3bab71-beeb-484b-af99-ea30fdef4773',
                               'name': 'Jeremy Ravenel',
                               'object': 'user',
                               'person': {'email': 'jeremy.ravenel@cashstory.com'},
                               'type': 'person'},
                'id': 'F%5D)%3F',
                'type': 'created_by'},
 'Last Edited Time': {'id': '0AiB',
                      'last_edited_time': '2021-08-23T13:11:00.000Z',
                      'type': 'last_edited_time'},
 'Meeting Date': {'date': {'end': None, 'start': '2021-05-23'},
                  'id': 'Q%5B%3Dk',
                  'type': 'date'},
 'Name': {'id': 'title',
          'title': [{'annotations': 

### Get content : Table format

In [122]:
df_content = get_content(PAGE_URL)
df_content

,object,id,type,has_children,created_time,last_edited_time,bold,italic,strikethrough,underline,code,color,content,link
0,block,d85d9955-cb2a-41dd-bf0a-3a051d1bb5de,unsupported,False,2021-05-18 17:17:00,2021-05-18 17:17:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,block,2d82db5e-5633-4ea6-9fc0-928b78fed69f,heading_1,False,2021-05-18 17:17:00,2021-05-18 17:17:00,False,False,False,False,False,default,What did we do last week?,NaN
2,block,02699496-f03c-46a9-b614-2a633a1752c9,bulleted_list_item,False,2021-05-18 17:17:00,2021-05-18 17:17:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,block,39ce79e0-9db4-4f5c-aa9f-a8ecb4fb3f11,heading_1,False,2021-05-18 17:17:00,2021-05-18 17:17:00,False,False,False,False,False,default,What are we doing this week?,NaN
4,block,6dadfa21-0d74-4bc0-9862-b90519eaeff8,bulleted_list_item,False,2021-05-18 17:17:00,2021-05-18 17:17:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,block,7553fb80-18d5-4065-b426-49c2ef27a9c1,heading_1,False,2021-05-18 17:17:00,2021-05-18 17:17:00,False,False,False,False,False,default,Potential blockers?,NaN
6,block,c1df621d-b8e5-4376-9c2f-f32e3a5a7153,bulleted_list_item,False,2021-05-18 17:17:00,2021-05-18 17:17:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,block,a49a16df-5837-47a1-830c-49ea96852cb1,heading_1,False,2021-05-18 17:17:00,2021-05-18 17:17:00,False,False,False,False,False,default,Action items,NaN
8,block,4a8ae2f8-0dde-4b3c-82ad-92d3729daffc,paragraph,False,2021-05-18 17:17:00,2021-05-18 17:17:00,False,False,False,False,False,default,followed by the name of a teammate to assign ...,NaN
9,block,4d77e9dc-d399-4a0b-8cc9-5c52b543e79d,to_do,False,2021-05-18 17:17:00,2021-05-18 17:17:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
